In [13]:
pwd

'/home/qibo/all_project/第三方/算话评估/auto_feat'

In [1]:
import random
import pandas as pd
import numpy as np
import random
import itertools
import lightgbm as lgb
from functools import reduce

data_path = './data/'
# test.csv  train.csv  train_target.csv
tra_x = pd.read_csv(data_path + '/train.csv')
tra_y = pd.read_csv(data_path + '/train_target.csv')
tes_x = pd.read_csv(data_path + '/test.csv')
final = tra_x.merge(tra_y,on='id')

final['certValidStop'] = final.certValidStop.astype(int)
final.fillna(-999,inplace=True)

tra_id = set(random.sample(range(final.shape[0]), 70000))
val_id = set(range(final.shape[0])) - tra_id
tra_id = [i for i in tra_id]
val_id = [i for i in val_id]
Train = final.iloc[tra_id,:].set_index(keys='id')
Valid = final.iloc[val_id,:].set_index(keys='id')
tra_x, tra_y = Train.drop('target', axis=1), Train.target
val_x, val_y = Valid.drop('target', axis=1), Valid.target

In [2]:
# 人工部分, 可以手工做出来 连续，rank, 离散， id;
simi_cols = [('certId', 'dist', 'residentAddr'), 
             ('certValidStop', 'certValidBegin'),
            ('highestEdu', 'edu'),
            ('certId2', 'dist2', 'residentAddr2')]


#小cate
zz = tra_x.apply(lambda x: len(x.unique())).sort_values(ascending=False)
small_cats = zz[zz<5].index.tolist()

# id 类，做拆分合并 
id_vars = ['bankCard', 'dist', 'residentAddr','certId']


# rank 部分，什么都不需要
rank_vars =['age', 'setupHour', 'weekday', 'basicLevel']


# 连续部分, 需要 RankCut [类别超过20个]
cont_vars = ['certValidStop', 'certValidBegin', 'lmt',]


# 离散 部分，需要 ToBadRatio+RankCut [类别超过20个]
disc_vars = ['ethnic', 'job', 'linkRela']


In [3]:
import sys
sys.path.append('../')
from auto_feat.Piplines import ToBadRatio_test, chiMerge_woe_test, RankCut_test, AutoCombine_test, EvalUtils_test

# 离散部分 变成rank
disc_vars = ['ethnic', 'job', 'linkRela']
TBR = ToBadRatio_test.ToBadRatio(cols=disc_vars)
TBR.fit(tra_x, tra_y)
tra_tbr = TBR.transform(tra_x)
val_tbr = TBR.transform(val_x)

# 连续部分 变成rank
cont_vars = ['certValidStop', 'certValidBegin', 'lmt',]
RC = RankCut_test.RankCut(cols=cont_vars, bins=20, null_value=-999, return_numeric=True)
RC.fit(tra_x)
tra_rc = RC.transform(tra_x, train=True)
val_rc = RC.transform(val_x, train=False)

# rank部分
rank_vars =['age', 'setupHour', 'weekday', 'basicLevel']


#id 类，可以TBR+RC 变成rank
id_vars = ['bankCard', 'dist', 'residentAddr','certId']
TBR2 = ToBadRatio_test.ToBadRatio(cols=id_vars)
TBR2.fit(tra_x, tra_y)
tra_id_tbr = TBR2.transform(tra_x)
val_id_tbr = TBR2.transform(val_x)
RC2 = RankCut_test.RankCut(cols=tra_id_tbr.columns, bins=20, null_value=-999, return_numeric=True)
RC2.fit(tra_id_tbr)
tra_id_tbr_rc = RC2.transform(tra_id_tbr, train=True)
val_id_tbr_rc = RC2.transform(val_id_tbr, train=False)

#id 类，可以缩小类别取前6位 然后TBR+RC 变成rank
id_vars = ['bankCard', 'dist', 'residentAddr','certId']
def id_reduce(ser, null_value=-999, divide=100):
    new_ser = ser.copy()
    new_ser[new_ser!=null_value] = (new_ser[new_ser!=null_value]/divide).astype(int)
    return new_ser
tra_x['residentAddr2'] = id_reduce(tra_x.residentAddr, divide=100)
val_x['residentAddr2'] = id_reduce(val_x.residentAddr, divide=100)
tra_x['certId2'] = id_reduce(tra_x.certId, divide=100)
val_x['certId2'] = id_reduce(val_x.certId, divide=100)
tra_x['dist2'] = id_reduce(tra_x.dist, divide=100)
val_x['dist2'] = id_reduce(val_x.dist, divide=100)
tra_x['bankCard2'] = id_reduce(tra_x.bankCard, divide=1000)
val_x['bankCard2'] = id_reduce(val_x.bankCard, divide=1000)
tra_x['bankCard3'] = id_reduce(tra_x.bankCard, divide=100000)
val_x['bankCard3'] = id_reduce(val_x.bankCard, divide=100000)
new_id_vars = ['bankCard2','bankCard3', 'dist2', 'residentAddr2','certId2']
TBR3 = ToBadRatio_test.ToBadRatio(cols=new_id_vars)
TBR3.fit(tra_x, tra_y)
tra_id2_tbr = TBR3.transform(tra_x)
val_id2_tbr = TBR3.transform(val_x)
RC3 = RankCut_test.RankCut(cols=tra_id2_tbr.columns, bins=20, null_value=-999, return_numeric=True)
RC3.fit(tra_id2_tbr)
tra_id2_tbr_rc = RC3.transform(tra_id2_tbr, train=True)
val_id2_tbr_rc = RC3.transform(val_id2_tbr, train=False)

TRA = tra_id2_tbr_rc.join(tra_id_tbr_rc).join(tra_rc).join(tra_tbr).join(tra_x[rank_vars])
VAL = val_id2_tbr_rc.join(val_id_tbr_rc).join(val_rc).join(val_tbr).join(val_x[rank_vars])

CM = chiMerge_woe_test.ChiMerge(cols=TRA.columns, null_value=-999, max_groups=10)
CM.fit(TRA, tra_y)
tra_cm = CM.transform(TRA)
val_cm = CM.transform(VAL)

WOE = chiMerge_woe_test.WOE_enc(cols=tra_cm.columns)
WOE.fit(tra_cm, tra_y)
tra_woe = WOE.transform(tra_cm)
val_woe = WOE.transform(val_cm)

Total time running function: fit with 0.05180621147155762 seconds
Total time running function: fit with 0.058130741119384766 seconds
Total time running function: fit with 0.14774680137634277 seconds
Total time running function: fit with 0.0973808765411377 seconds
Total time running function: fit with 0.08989429473876953 seconds
Total time running function: fit with 0.08724093437194824 seconds


../auto_feat/Piplines/chiMerge_woe_test.py:86: RuntimeWarning: invalid value encountered in true_divide
  square = (arr-E)**2 / E


In [4]:
AC = AutoCombine_test.AutoCombine(cols=tra_cm.columns, order=2, null_value=-999)
AC.fit(tra_cm)
tra_ac = AC.transform(tra_cm, True)
val_ac = AC.transform(val_cm, False)
TBR3 = ToBadRatio_test.ToBadRatio(cols=tra_ac.columns)
TBR3.fit(tra_ac, tra_y)
tra_ac_tbr = TBR3.transform(tra_ac)
val_ac_tbr = TBR3.transform(val_ac)
RC3 = RankCut_test.RankCut(cols=tra_ac_tbr.columns, bins=20, null_value=-999, return_numeric=True)
RC3.fit(tra_ac_tbr)
tra_ac_tbr_rc = RC3.transform(tra_ac_tbr, train=True)
val_ac_tbr_rc = RC3.transform(val_ac_tbr, train=False)

processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(bankCard3)))), 0/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(dist2)))), 1/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(residentAddr2)))), 2/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(certId2)))), 3/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(bankCard)))), 4/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(dist)))), 5/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(residentAddr)))), 6/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(TBR(certId)))), 7/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(certValidStop))), 8/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(certValidBegin))), 9/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(RC(lmt))), 10/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(TBR(ethnic))), 11/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(TBR(job))), 12/171
processing col: comb(CM(RC(TBR(bankCard2))),CM(TBR(linkRela

processing col: comb(CM(RC(TBR(certId))),CM(RC(lmt))), 118/171
processing col: comb(CM(RC(TBR(certId))),CM(TBR(ethnic))), 119/171
processing col: comb(CM(RC(TBR(certId))),CM(TBR(job))), 120/171
processing col: comb(CM(RC(TBR(certId))),CM(TBR(linkRela))), 121/171
processing col: comb(CM(RC(TBR(certId))),CM(age)), 122/171
processing col: comb(CM(RC(TBR(certId))),CM(setupHour)), 123/171
processing col: comb(CM(RC(TBR(certId))),CM(weekday)), 124/171
processing col: comb(CM(RC(TBR(certId))),CM(basicLevel)), 125/171
processing col: comb(CM(RC(certValidStop)),CM(RC(certValidBegin))), 126/171
processing col: comb(CM(RC(certValidStop)),CM(RC(lmt))), 127/171
processing col: comb(CM(RC(certValidStop)),CM(TBR(ethnic))), 128/171
processing col: comb(CM(RC(certValidStop)),CM(TBR(job))), 129/171
processing col: comb(CM(RC(certValidStop)),CM(TBR(linkRela))), 130/171
processing col: comb(CM(RC(certValidStop)),CM(age)), 131/171
processing col: comb(CM(RC(certValidStop)),CM(setupHour)), 132/171
processin

In [8]:
WOE.woe_dics['CM(RC(TBR(certId)))']

{'woe': {0.0: -16.915125082734995,
  '[0.0,12.0)': -19.24758085635012,
  '[12.0,13.0)': -16.59708960428875,
  '[13.0,14.0)': -16.052425872905555,
  '[14.0,15.0)': -17.14971558447376,
  '[15.0,16.0)': -16.97865908342596,
  '[16.0,17.0)': -17.094485991555445,
  '[17.0,18.0)': -0.07904688572629442,
  '[18.0,19.0)': 1.2216737544829963,
  '[19.0,999.0)': 2.3422900580807657,
  '[999.0,_)': -11.397672269268945},
 'iv': 13.949470765959651}

In [9]:
def _auc_impo(tra_x, tra_y, val_x, val_y, params=None, verbose=1):
    print(' train:{}   valid:{}'.format(tra_x.shape, val_x.shape))
    if params is None:
        params = {'task': 'train',
              'boosting_type': 'gbdt',
              'objective': 'binary',
              'metric': 'auc',
              'num_threads': 10,
              'num_leaves': 31,  # 31,
              'learning_rate': 0.001,  # 0.002
              'feature_fraction': 0.5,
              'lambda_l2': 100,
              'bagging_fraction': 0.5,
              'bagging_freq': 5}

    cv_train = lgb.Dataset(tra_x, tra_y.astype('int'))        
    cv_valid = lgb.Dataset(val_x, val_y.astype('int'))        
    gbm = lgb.train(params,       # 参数字典
                    cv_train,       # 训练集
                    num_boost_round=2000,       # 迭代次数
                    valid_sets=cv_train,        # 验证集
                    early_stopping_rounds = 100,
                    verbose_eval=verbose)
#     print('with best auc:{}'.format(gbm.best_score['valid_0']))
    return gbm

In [10]:
_auc_impo(tra_x, tra_y, val_x, val_y)

 train:(70000, 108)   valid:(62029, 108)
[1]	training's auc: 0.665507
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.688484
[3]	training's auc: 0.704758
[4]	training's auc: 0.720927
[5]	training's auc: 0.717592
[6]	training's auc: 0.717435
[7]	training's auc: 0.722932
[8]	training's auc: 0.723013
[9]	training's auc: 0.721797
[10]	training's auc: 0.722271
[11]	training's auc: 0.720294
[12]	training's auc: 0.717834
[13]	training's auc: 0.71935
[14]	training's auc: 0.722581
[15]	training's auc: 0.720971
[16]	training's auc: 0.719385
[17]	training's auc: 0.720098
[18]	training's auc: 0.721145
[19]	training's auc: 0.719268
[20]	training's auc: 0.71991
[21]	training's auc: 0.720269
[22]	training's auc: 0.720589
[23]	training's auc: 0.721477
[24]	training's auc: 0.720528
[25]	training's auc: 0.72121
[26]	training's auc: 0.721034
[27]	training's auc: 0.720844
[28]	training's auc: 0.720004
[29]	training's auc: 0.719905
[30]	training's auc: 0.719991
[31]	tra

In [11]:
_auc_impo(tra_x.join(TRA, lsuffix='_'), tra_y, val_x.join(VAL, lsuffix='_'), val_y)

 train:(70000, 127)   valid:(62029, 127)
[1]	training's auc: 0.882301
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.899366
[3]	training's auc: 0.973468
[4]	training's auc: 0.972784
[5]	training's auc: 0.970397
[6]	training's auc: 0.969657
[7]	training's auc: 0.969629
[8]	training's auc: 0.976066
[9]	training's auc: 0.978299
[10]	training's auc: 0.980941
[11]	training's auc: 0.982739
[12]	training's auc: 0.981895
[13]	training's auc: 0.98227
[14]	training's auc: 0.982205
[15]	training's auc: 0.982309
[16]	training's auc: 0.982316
[17]	training's auc: 0.982891
[18]	training's auc: 0.982871
[19]	training's auc: 0.983397
[20]	training's auc: 0.983397
[21]	training's auc: 0.983198
[22]	training's auc: 0.982652
[23]	training's auc: 0.982289
[24]	training's auc: 0.981774
[25]	training's auc: 0.981057
[26]	training's auc: 0.979952
[27]	training's auc: 0.979717
[28]	training's auc: 0.979415
[29]	training's auc: 0.980617
[30]	training's auc: 0.980351
[31]	t

In [12]:
_auc_impo(tra_ac_tbr_rc, tra_y, val_ac_tbr_rc, val_y)

 train:(70000, 171)   valid:(62029, 171)
[1]	training's auc: 0.977876
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.983144
[3]	training's auc: 0.983722
[4]	training's auc: 0.98458
[5]	training's auc: 0.98482
[6]	training's auc: 0.987776
[7]	training's auc: 0.987893
[8]	training's auc: 0.987905
[9]	training's auc: 0.988191
[10]	training's auc: 0.988301
[11]	training's auc: 0.988433
[12]	training's auc: 0.988376
[13]	training's auc: 0.988414
[14]	training's auc: 0.988383
[15]	training's auc: 0.988381
[16]	training's auc: 0.988552
[17]	training's auc: 0.988637
[18]	training's auc: 0.98868
[19]	training's auc: 0.98864
[20]	training's auc: 0.988638
[21]	training's auc: 0.988753
[22]	training's auc: 0.988785
[23]	training's auc: 0.988808
[24]	training's auc: 0.988895
[25]	training's auc: 0.988938
[26]	training's auc: 0.988914
[27]	training's auc: 0.988866
[28]	training's auc: 0.988861
[29]	training's auc: 0.988852
[30]	training's auc: 0.988862
[31]	trai

[272]	training's auc: 0.990075
[273]	training's auc: 0.990122
[274]	training's auc: 0.990121
[275]	training's auc: 0.99011
[276]	training's auc: 0.990113
[277]	training's auc: 0.990112
[278]	training's auc: 0.990113
[279]	training's auc: 0.990113
[280]	training's auc: 0.990114
[281]	training's auc: 0.990115
[282]	training's auc: 0.990114
[283]	training's auc: 0.990115
[284]	training's auc: 0.990131
[285]	training's auc: 0.990131
[286]	training's auc: 0.990132
[287]	training's auc: 0.990133
[288]	training's auc: 0.990127
[289]	training's auc: 0.99013
[290]	training's auc: 0.990128
[291]	training's auc: 0.990129
[292]	training's auc: 0.990127
[293]	training's auc: 0.990126
[294]	training's auc: 0.990126
[295]	training's auc: 0.990123
[296]	training's auc: 0.990123
[297]	training's auc: 0.990125
[298]	training's auc: 0.990124
[299]	training's auc: 0.990126
[300]	training's auc: 0.990127
[301]	training's auc: 0.990124
[302]	training's auc: 0.990122
[303]	training's auc: 0.99012
[304]	train

[538]	training's auc: 0.99057
[539]	training's auc: 0.990568
[540]	training's auc: 0.990569
[541]	training's auc: 0.990566
[542]	training's auc: 0.990567
[543]	training's auc: 0.990566
[544]	training's auc: 0.990566
[545]	training's auc: 0.990566
[546]	training's auc: 0.990566
[547]	training's auc: 0.990566
[548]	training's auc: 0.990569
[549]	training's auc: 0.99058
[550]	training's auc: 0.99058
[551]	training's auc: 0.99058
[552]	training's auc: 0.990581
[553]	training's auc: 0.990581
[554]	training's auc: 0.990577
[555]	training's auc: 0.990574
[556]	training's auc: 0.990574
[557]	training's auc: 0.990573
[558]	training's auc: 0.990572
[559]	training's auc: 0.990571
[560]	training's auc: 0.99057
[561]	training's auc: 0.990567
[562]	training's auc: 0.99069
[563]	training's auc: 0.990704
[564]	training's auc: 0.990702
[565]	training's auc: 0.990703
[566]	training's auc: 0.990701
[567]	training's auc: 0.9907
[568]	training's auc: 0.990699
[569]	training's auc: 0.990697
[570]	training's

[823]	training's auc: 0.991107
[824]	training's auc: 0.991109
[825]	training's auc: 0.991107
[826]	training's auc: 0.991105
[827]	training's auc: 0.991105
[828]	training's auc: 0.991112
[829]	training's auc: 0.991117
[830]	training's auc: 0.991123
[831]	training's auc: 0.991124
[832]	training's auc: 0.991124
[833]	training's auc: 0.991129
[834]	training's auc: 0.991131
[835]	training's auc: 0.991131
[836]	training's auc: 0.991136
[837]	training's auc: 0.991139
[838]	training's auc: 0.99114
[839]	training's auc: 0.991141
[840]	training's auc: 0.991154
[841]	training's auc: 0.991154
[842]	training's auc: 0.991156
[843]	training's auc: 0.991159
[844]	training's auc: 0.991157
[845]	training's auc: 0.991156
[846]	training's auc: 0.991154
[847]	training's auc: 0.991156
[848]	training's auc: 0.991155
[849]	training's auc: 0.991155
[850]	training's auc: 0.991155
[851]	training's auc: 0.991154
[852]	training's auc: 0.991154
[853]	training's auc: 0.991152
[854]	training's auc: 0.991149
[855]	tra

[1111]	training's auc: 0.991427
[1112]	training's auc: 0.991429
[1113]	training's auc: 0.991427
[1114]	training's auc: 0.991429
[1115]	training's auc: 0.99143
[1116]	training's auc: 0.99143
[1117]	training's auc: 0.99143
[1118]	training's auc: 0.991429
[1119]	training's auc: 0.991429
[1120]	training's auc: 0.991429
[1121]	training's auc: 0.991433
[1122]	training's auc: 0.991441
[1123]	training's auc: 0.991444
[1124]	training's auc: 0.991444
[1125]	training's auc: 0.991447
[1126]	training's auc: 0.991446
[1127]	training's auc: 0.991452
[1128]	training's auc: 0.991453
[1129]	training's auc: 0.991456
[1130]	training's auc: 0.991459
[1131]	training's auc: 0.991461
[1132]	training's auc: 0.991461
[1133]	training's auc: 0.991461
[1134]	training's auc: 0.991462
[1135]	training's auc: 0.991461
[1136]	training's auc: 0.991464
[1137]	training's auc: 0.991465
[1138]	training's auc: 0.991471
[1139]	training's auc: 0.991472
[1140]	training's auc: 0.991471
[1141]	training's auc: 0.991473
[1142]	trai

[1393]	training's auc: 0.99201
[1394]	training's auc: 0.992011
[1395]	training's auc: 0.992013
[1396]	training's auc: 0.992014
[1397]	training's auc: 0.992014
[1398]	training's auc: 0.992014
[1399]	training's auc: 0.992016
[1400]	training's auc: 0.992017
[1401]	training's auc: 0.992018
[1402]	training's auc: 0.992019
[1403]	training's auc: 0.99202
[1404]	training's auc: 0.992022
[1405]	training's auc: 0.992023
[1406]	training's auc: 0.992025
[1407]	training's auc: 0.992024
[1408]	training's auc: 0.992024
[1409]	training's auc: 0.992023
[1410]	training's auc: 0.992024
[1411]	training's auc: 0.992025
[1412]	training's auc: 0.992024
[1413]	training's auc: 0.992022
[1414]	training's auc: 0.992023
[1415]	training's auc: 0.992024
[1416]	training's auc: 0.992025
[1417]	training's auc: 0.992025
[1418]	training's auc: 0.992025
[1419]	training's auc: 0.99203
[1420]	training's auc: 0.99203
[1421]	training's auc: 0.99203
[1422]	training's auc: 0.99203
[1423]	training's auc: 0.99203
[1424]	training

[1687]	training's auc: 0.992493
[1688]	training's auc: 0.992493
[1689]	training's auc: 0.992492
[1690]	training's auc: 0.992491
[1691]	training's auc: 0.992494
[1692]	training's auc: 0.992493
[1693]	training's auc: 0.992498
[1694]	training's auc: 0.992499
[1695]	training's auc: 0.992499
[1696]	training's auc: 0.992499
[1697]	training's auc: 0.992504
[1698]	training's auc: 0.992505
[1699]	training's auc: 0.99251
[1700]	training's auc: 0.992513
[1701]	training's auc: 0.992517
[1702]	training's auc: 0.992524
[1703]	training's auc: 0.992528
[1704]	training's auc: 0.992529
[1705]	training's auc: 0.992533
[1706]	training's auc: 0.992538
[1707]	training's auc: 0.992544
[1708]	training's auc: 0.992549
[1709]	training's auc: 0.992555
[1710]	training's auc: 0.99256
[1711]	training's auc: 0.992559
[1712]	training's auc: 0.99256
[1713]	training's auc: 0.99256
[1714]	training's auc: 0.992561
[1715]	training's auc: 0.992564
[1716]	training's auc: 0.992562
[1717]	training's auc: 0.992563
[1718]	train

[1953]	training's auc: 0.992886
[1954]	training's auc: 0.992888
[1955]	training's auc: 0.992888
[1956]	training's auc: 0.992889
[1957]	training's auc: 0.992889
[1958]	training's auc: 0.99289
[1959]	training's auc: 0.992897
[1960]	training's auc: 0.992896
[1961]	training's auc: 0.992899
[1962]	training's auc: 0.9929
[1963]	training's auc: 0.992901
[1964]	training's auc: 0.992902
[1965]	training's auc: 0.992905
[1966]	training's auc: 0.992903
[1967]	training's auc: 0.992906
[1968]	training's auc: 0.992905
[1969]	training's auc: 0.992907
[1970]	training's auc: 0.992909
[1971]	training's auc: 0.99291
[1972]	training's auc: 0.992912
[1973]	training's auc: 0.992914
[1974]	training's auc: 0.992917
[1975]	training's auc: 0.992918
[1976]	training's auc: 0.992921
[1977]	training's auc: 0.992922
[1978]	training's auc: 0.992923
[1979]	training's auc: 0.992925
[1980]	training's auc: 0.992926
[1981]	training's auc: 0.992927
[1982]	training's auc: 0.99293
[1983]	training's auc: 0.99293
[1984]	trainin

In [8]:
import pandas as pd
import numpy as np
import random
import itertools
import lightgbm as lgb
from functools import reduce
from utils import merge_box
import sys
sys.path.append('../')
from utils import _merge_box, k_order_comb
import itertools

class autofeat_draft:
    '''
    1.  k_order_comb generation 特征组合生成
    1.5. merge_box 组合特征后可以接一下merge_box
    2.  dummy_lgb_topk 特征降维
    3.  auto_transform 小微特征加减乘 生成
    '''
    def __init__(self, global_id = 'id'):
        self.global_id = global_id
    
    def k_order_comb(self, tra, val, column_names, order, merge_tail, thresh):
        assert tra.index.name == self.global_id
        return k_order_comb(tra, val, column_names, order, merge_tail, thresh)
    
    def merge_box(self, tra, val, columns, thresh):
        return merge_box(tra, val, columns, thresh)
    
    
    def dummy_lgb_topk(self, tra_x, tra_y, val_x, val_y, col_names_list, max_unique, top_k):
        # 每囤够max_unique个dummy 就跑一次 lgb top k 
        tmp_tra_df = pd.DataFrame(index = tra_x.index)
        tmp_val_df = pd.DataFrame(index = val_x.index)
        final_tra, final_val, unique_num = [], [], 0
        for idx, col in enumerate(col_names_list):
            print('dummy column: {}/{}'.format(idx, len(col_names_list)))
            unique_num += len(tra_x[col].unique())
            if unique_num <= max_unique + top_k*idx: # 因为是滚动式的，每提取出一次 dummy_lgb_topk 就在此基础上；
                tmp_tra_df = pd.get_dummies(tra_x[col], prefix=col+'_').join(tmp_tra_df)
                tmp_val_df = pd.get_dummies(val_x[col], prefix=col+'_').join(tmp_val_df)
                tmp_tra_df = tmp_tra_df[tmp_val_df.columns] # 防止 tra, val 列数因为dummy 不一致
            else:
                print('dummy reach: {}/{}, take only: {} combinations'.format(unique_num, max_unique, top_k))
                tra_1, val_1, col_names, gbm = self._auto_topk(tmp_tra_df, tra_y, tmp_val_df, val_y, top_k)
                final_tra.append(tra_1)
                final_val.append(val_1)
                unique_num = 0
                tmp_tra_df = tra_1
                tmp_val_df = val_1
                assert tmp_val_df.shape[1] == top_k
        if len(final_tra)==0:
            tra_1, val_1, col_names, gbm = self._auto_topk(tmp_tra_df, tra_y, tmp_val_df, val_y, top_k)
            final_tra.append(tra_1)
            final_val.append(val_1)
        return pd.concat(final_tra, axis=1), pd.concat(final_val, axis=1)
    
    def auto_transform(self, tra, val, column_names, order=2, op='add'):
        assert op in ['add', 'multiply', 'sub']
        assert tra.index.name == self.global_id == val.index.name
        DF_tra, DF_val = pd.DataFrame(index=tra.index), pd.DataFrame(index=val.index)
        col_names_list = [i for i in itertools.combinations(column_names, order)]
        for col_names in col_names_list:
            tra_cols = [tra[i] for i in col_names]
            val_cols = [val[i] for i in col_names]
            new_name = '{}('.format(op) + ','.join(col_names) + ')'
            if op=='add':
                DF_tra[new_name] = reduce(lambda x, y: x+y, tra_cols)
                DF_val[new_name] = reduce(lambda x, y: x+y, val_cols)           
            elif op =='multiply':
                DF_tra[new_name] = reduce(lambda x, y: x*y, tra_cols)
                DF_val[new_name] = reduce(lambda x, y: x*y, val_cols)
            elif op == 'sub':
                DF_tra[new_name] = reduce(lambda x, y: x-y, tra_cols)
                DF_val[new_name] = reduce(lambda x, y: x-y, val_cols)
            else:
                raise Exception('op error')
        return DF_tra, DF_val

    
    def _auto_topk(self, tra_x, tra_y, val_x, val_y, top_k):
        gbm = self._auc_impo(tra_x, tra_y, val_x, val_y)
        tra_1, col_names = self._topk_feat(tra_x, gbm, top_k)
        val_1 = self._topk_feat_transfrom(val_x, col_names)
        return tra_1, val_1, col_names, gbm
    
    def _auc_impo(self, tra_x, tra_y, val_x, val_y, params=None, verbose=0):
        print(' train:{}   valid:{}'.format(tra_x.shape, val_x.shape))
        if params is None:
            params = {'task': 'train',
                  'boosting_type': 'gbdt',
                  'objective': 'binary',
                  'metric': 'auc',
                  'num_threads': 10,
                  'num_leaves': 3,  # 31,
                  'learning_rate': 0.008,  # 0.002
                  'feature_fraction': 0.5,
                  'lambda_l2': 140,
                  'bagging_fraction': 0.5,
                  'bagging_freq': 5}

        cv_train = lgb.Dataset(tra_x, tra_y.astype('int'))        
        cv_valid = lgb.Dataset(val_x, val_y.astype('int'))        
        gbm = lgb.train(params,       # 参数字典
                        cv_train,       # 训练集
                        num_boost_round=2000,       # 迭代次数
                        valid_sets=cv_valid,        # 验证集
                        early_stopping_rounds = 100,
                        verbose_eval=verbose)
        print('with best auc:{}'.format(gbm.best_score['valid_0']))
        return gbm

    def _topk_feat(self, tra_x, gbm, k, importance_type='gain'):
        assert importance_type in ['split', 'gain']
        z = pd.DataFrame()
        z['col'] = tra_x.columns
        z['imp'] = gbm.feature_importance(importance_type) 
        col_names = z.nlargest(k, 'imp').col.tolist()
        return tra_x[col_names], col_names

    def _topk_feat_transfrom(self, val_x, col_names):
        return val_x[col_names]


    
class PrePostProcess:
    '''  
    1. rank_cut 排序分箱；
    2. null 处理; 造两列空置率； 
    3. to be done: 根据分布一致性删除列；
    4. merge_box: 合箱子；
    '''   
    
    def group_simi_cols(self, tra_x, val_x, simi_cols):
        tra_all, val_all = [], []
        for i in simi_cols:
            tra_tmp, val_tmp= self.auto_transform(tra_x, val_x, column_names=list(i), order=2, op='sub')
            tra_all.append(tra_tmp)
            val_all.append(val_tmp)
        return pd.concat(tra_all, axis=1), pd.concat(val_all, axis=1)
    
    def _to_bins(self, col, q, labels=None):
        new_col, bins = pd.cut(col, bins=q, labels=labels, retbins=True)
        return new_col, bins

    def _to_bins_trans(self, col, bins, labels=None):
        new_col, bins = pd.cut(col, bins=bins, retbins=True, labels=labels)
        return new_col, bins

    def add_null_rate(self, df, null_val):
        assert 'null_num' not in df.columns
        new_df = pd.DataFrame(index=df.index)
        null_df = new_df==null_val
        new_df['null_num'] = null_df.sum(axis=1)
        new_df['null_rate'] = null_df.sum(axis=1)/null_df.shape[1]
        return new_df
    
    def rank_cut(self, tra, val, cont_vars):
        #  列 rank_cut
        tmp_tra, tmp_val = pd.DataFrame(index=tra.index), pd.DataFrame(index=val.index)
        for col_name in cont_vars:
            a1, bins1 = self._to_bins(tra[col_name], q=10, labels=range(10))
            assert col_name+'_rankcut' not in tra.columns
            tmp_tra[col_name+'_rankcut'] = a1
            b1, _ = self._to_bins_trans(val[col_name], bins=bins1, labels=range(10))
            tmp_val[col_name+'_rankcut'] = b1
        return tmp_tra, tmp_val
    

    def merge_box(self, tra, val, columns, thresh):
        return merge_box(tra, val, columns, thresh)

    def auto_transform(self, tra, val, column_names, order=2, op='add'):
        # 确认输入是否都是cat
        assert op in ['add', 'multiply', 'sub']
    #     assert tra.index.name == self.global_id == val.index.name
        DF_tra, DF_val = pd.DataFrame(index=tra.index), pd.DataFrame(index=val.index)
        col_names_list = [i for i in itertools.combinations(column_names, order)]
        for col_names in col_names_list:
            tra_cols = [tra[i] for i in col_names]
            val_cols = [val[i] for i in col_names]
            new_name = '{}('.format(op) + ','.join(col_names) + ')'
            if op=='add':
                DF_tra[new_name] = reduce(lambda x, y: x+y, tra_cols)
                DF_val[new_name] = reduce(lambda x, y: x+y, val_cols)           
            elif op =='multiply':
                DF_tra[new_name] = reduce(lambda x, y: x*y, tra_cols)
                DF_val[new_name] = reduce(lambda x, y: x*y, val_cols)
            elif op == 'sub':
                DF_tra[new_name] = reduce(lambda x, y: x-y, tra_cols)
                DF_val[new_name] = reduce(lambda x, y: x-y, val_cols)
            else:
                raise Exception('op error')
        return DF_tra, DF_val




In [4]:
PPP = PrePostProcess()
FFF = autofeat_draft()

In [5]:
a, b = FFF.k_order_comb(tra_x, val_x, column_names=tra_x.columns[:4], order=2, thresh=0.7, merge_tail=True)

after 2 order feature combination, there are totally 6 new features generated


In [ ]:
FFF
##### ##### ##### ##### ##### #####
    ------------------    
    auto_transform
    ------------------    
    k_order_comb
    ------------------
    dummy_lgb_topk
        _auto_topk
            _auc_impo
            _topk_feat
            _topk_feat_transfrom

        
PPP        
##### ##### ##### ##### ##### #####          
    ------------------
    group_simi_cols
        auto_transform
    ------------------
    rank_cut
        _to_bins
        _to_bins_trans


In [14]:


import collections
import pandas as pd
import numpy as np
import lightgbm as lgb
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
import random
import math
# import featuretools as ft
import heapq
import itertools
from featexp import get_univariate_plots,get_trend_stats
warnings.filterwarnings("ignore")
# %matplotlib inline
import pickle
sys.path.append('../')
from auto_feat.AutoFeat import PrePostProcess, autofeat_draft
from auto_feat import HumanLook
# import ipdb
# from ipdb import set_trace


In [2]:
# !python -m pip install featuretools
# !pip install featexp
# !pip install ipdb

In [3]:
data_path = '/home/chenxiaotian/Projects/xiamen_match/data'
# test.csv  train.csv  train_target.csv
tra_x = pd.read_csv(data_path + '/train.csv')
tra_y = pd.read_csv(data_path + '/train_target.csv')
final = tra_x.merge(tra_y,on='id')
final['dist']= final.dist.apply(lambda x: int(x/100))
final['certId'] = final.certId.apply(lambda x: int(x/100))
final['residentAddr'] = final.residentAddr.apply(lambda x: int(x/100))
final['certValidStop'] = final.certValidStop.astype(int)

tra_id = set(random.sample(range(final.shape[0]), 70000))
val_id = set(range(final.shape[0])) - tra_id
tra_id = [i for i in tra_id]
val_id = [i for i in val_id]
Train = final.iloc[tra_id,:]
Valid = final.iloc[val_id,:]
tra_x, tra_y = Train.drop('target', axis=1).set_index(keys='id'), Train[['id','target']].set_index(keys='id')
val_x, val_y = Valid.drop('target', axis=1).set_index(keys='id'), Valid[['id','target']].set_index(keys='id')

tra_x['dist2']= tra_x.dist.apply(lambda x: int(x/100))
val_x['dist2']= val_x.dist.apply(lambda x: int(x/100))

tra_x['certId2']= tra_x.certId.apply(lambda x: int(x/100))
val_x['certId2']= val_x.certId.apply(lambda x: int(x/100))

tra_x['residentAddr2']= tra_x.dist.apply(lambda x: int(x/100))
val_x['residentAddr2']= val_x.dist.apply(lambda x: int(x/100))


In [4]:
rank_cols = ['lmt', 'age'] # 大于10类的有排序含义的特征
# rank_cut 降cat

mid_cats = ['ethnic', 'job'] # 大于10类的, 小于50，无排序的特征；
# 手工 降cat + log 分箱; 可以写个自动组合箱子

big_cats = ['certId', 'certId2', 'dist', 'residentAddr', 'dist2', 'residentAddr2'] # 大于50无排序特征；
# 手工 降cat + log 分箱

time_cols = ['setupHour', 'certValidStop', 'certValidBegin', 'weekday'] 
# 手工降cat

simi_cols = [('certId', 'dist', 'residentAddr'), 
             ('certValidStop', 'certValidBegin'),
            ('highestEdu', 'edu'),
            ('certId2', 'dist2', 'residentAddr2')]

PPP = PrePostProcess()
FFF = autofeat_draft()
TRAIN_FEAT, VALID_FEAT = [], []
# rank_cut 降cat
rank_tra, rank_val = PPP.rank_cut(tra_x, val_x, rank_cols)

# 手工 降cat + log 分箱
mergebox_tra, mergebox_val = PPP.merge_box(tra_x, val_x, mid_cats+big_cats, thresh=0.92)

# simi_cols 做DIFF 变成连续型 然后分箱子
simi_tra, simi_val = PPP.group_simi_cols(tra_x, val_x, simi_cols)
rank_simi_tra, rank_simi_val = PPP.rank_cut(simi_tra, simi_val, simi_tra.columns)

# 时间分箱子
time_tra,  time_val = PPP.rank_cut(tra_x, val_x, time_cols)

TRAIN_FEAT.extend([rank_tra, mergebox_tra, simi_tra, rank_simi_tra, time_tra, tra_x])
VALID_FEAT.extend([rank_val, mergebox_val, simi_val, rank_simi_val, time_val, val_x])

aaa = pd.concat(TRAIN_FEAT, axis=1)
bbb = pd.concat(VALID_FEAT, axis=1)

In [5]:
# FFF._auc_impo(tra_x, tra_y, val_x, val_y)

In [6]:
# HumanLook._col_unique(aaa)[HumanLook._col_unique(aaa)>30].index.tolist()

In [7]:
# # 核心是什么？
#     # 是涨缩
#     - 涨： auto_combine & auto_transform  评估一下
#     - 缩： merge_box & dummy_lgb_topk 评估一下

In [8]:
ALL_Features = dict()
ALL_Features['train'] = dict()
ALL_Features['valid'] = dict()

tra_x, tra_y, val_x, val_y, aff_cls, add_merge_box = aaa, tra_y, bbb, val_y, FFF, False
ALL_Features['train']['origin']= tra_x
ALL_Features['valid']['origin']= val_x

for order in range(2,5):
    print('--------------------------order:{}--------------------'.format(order))
    cols = random.sample(tra_x.columns.tolist(), tra_x.shape[1])
    tra_x, val_x = aff_cls.k_order_comb(tra_x, val_x, column_names=cols, order=order)

    if add_merge_box:
        tra_x, val_x =  aff_cls.merge_box(tra_x, val_x, columns=tra_x.columns, thresh=0.92)
        print('eval with only lgb_top_k feats after k_order_comb & dummy lgb & merge_box with order={}'.format(order))
        _ = aff_cls._auc_impo(tra_x.values, tra_y, val_x.values, val_y)


    cols = tra_x.columns.tolist()
    tra_x_lgbk, val_x_lgbk = aff_cls.dummy_lgb_topk(tra_x, tra_y, val_x, val_y, 
                                  col_names_list=cols, max_unique=20000, top_k=300)

    print('| dummy_lgb_topk | k_order_comb | add_merge_box={}'.format(add_merge_box))
    _ = aff_cls._auc_impo(tra_x_lgbk.values, tra_y, val_x_lgbk.values, val_y)


   # 5. feature_dfs. append(new_cats)
    ALL_Features['train']['order_{}'.format(order)]= tra_x
    ALL_Features['valid']['order_{}'.format(order)]= val_x       
    with open('./feats_dict_order{}.pkl'.format(order), 'wb') as f:
        pickle.dump(ALL_Features, f, pickle.HIGHEST_PROTOCOL)


--------------------------order:2--------------------
after 2 order feature combination, there are totally 9180 new features generated
dummy column: 0/9180
dummy column: 1/9180
dummy column: 2/9180
dummy column: 3/9180
dummy column: 4/9180
dummy column: 5/9180
dummy column: 6/9180
dummy column: 7/9180
dummy column: 8/9180
dummy column: 9/9180
dummy column: 10/9180
dummy column: 11/9180
dummy column: 12/9180
dummy column: 13/9180
dummy column: 14/9180
dummy column: 15/9180
dummy column: 16/9180
dummy column: 17/9180
dummy column: 18/9180
dummy column: 19/9180
dummy column: 20/9180
dummy column: 21/9180
dummy column: 22/9180
dummy column: 23/9180
dummy column: 24/9180
dummy column: 25/9180
dummy column: 26/9180
dummy column: 27/9180
dummy column: 28/9180
dummy column: 29/9180
dummy column: 30/9180
dummy column: 31/9180
dummy column: 32/9180
dummy column: 33/9180
dummy column: 34/9180
dummy column: 35/9180
dummy column: 36/9180
dummy column: 37/9180
dummy column: 38/9180
dummy column: 39/

dummy column: 356/9180
dummy column: 357/9180
dummy column: 358/9180
dummy column: 359/9180
dummy column: 360/9180
dummy column: 361/9180
dummy column: 362/9180
dummy column: 363/9180
dummy column: 364/9180
dummy column: 365/9180
dummy column: 366/9180
dummy column: 367/9180
dummy column: 368/9180
dummy column: 369/9180
dummy column: 370/9180
dummy column: 371/9180
dummy column: 372/9180
dummy column: 373/9180
dummy column: 374/9180
dummy column: 375/9180
dummy column: 376/9180
dummy column: 377/9180
dummy column: 378/9180
dummy column: 379/9180
dummy column: 380/9180
dummy column: 381/9180
dummy column: 382/9180
dummy column: 383/9180
dummy column: 384/9180
dummy column: 385/9180
dummy column: 386/9180
dummy column: 387/9180
dummy column: 388/9180
dummy column: 389/9180
dummy column: 390/9180
dummy column: 391/9180
dummy column: 392/9180
dummy column: 393/9180
dummy column: 394/9180
dummy column: 395/9180
dummy column: 396/9180
dummy column: 397/9180
dummy column: 398/9180
dummy colum

dummy column: 713/9180
dummy column: 714/9180
dummy column: 715/9180
dummy column: 716/9180
dummy column: 717/9180
dummy column: 718/9180
dummy column: 719/9180
dummy column: 720/9180
dummy column: 721/9180
dummy column: 722/9180
dummy column: 723/9180
dummy column: 724/9180
dummy column: 725/9180
dummy column: 726/9180
dummy column: 727/9180
dummy column: 728/9180
dummy column: 729/9180
dummy column: 730/9180
dummy column: 731/9180
dummy column: 732/9180
dummy column: 733/9180
dummy column: 734/9180
dummy column: 735/9180
dummy column: 736/9180
dummy column: 737/9180
dummy column: 738/9180
dummy column: 739/9180
dummy column: 740/9180
dummy column: 741/9180
dummy column: 742/9180
dummy column: 743/9180
dummy column: 744/9180
dummy column: 745/9180
dummy column: 746/9180
dummy column: 747/9180
dummy column: 748/9180
dummy column: 749/9180
dummy column: 750/9180
dummy column: 751/9180
dummy column: 752/9180
dummy column: 753/9180
dummy column: 754/9180
dummy column: 755/9180
dummy colum

dummy column: 1067/9180
dummy column: 1068/9180
dummy column: 1069/9180
dummy column: 1070/9180
dummy column: 1071/9180
dummy column: 1072/9180
dummy column: 1073/9180
dummy column: 1074/9180
dummy column: 1075/9180
dummy column: 1076/9180
dummy column: 1077/9180
dummy column: 1078/9180
dummy column: 1079/9180
dummy column: 1080/9180
dummy column: 1081/9180
dummy column: 1082/9180
dummy column: 1083/9180
dummy column: 1084/9180
dummy column: 1085/9180
dummy column: 1086/9180
dummy column: 1087/9180
dummy column: 1088/9180
dummy column: 1089/9180
dummy column: 1090/9180
dummy column: 1091/9180
dummy column: 1092/9180
dummy column: 1093/9180
dummy column: 1094/9180
dummy column: 1095/9180
dummy column: 1096/9180
dummy column: 1097/9180
dummy column: 1098/9180
dummy column: 1099/9180
dummy column: 1100/9180
dummy column: 1101/9180
dummy column: 1102/9180
dummy column: 1103/9180
dummy column: 1104/9180
dummy column: 1105/9180
dummy column: 1106/9180
dummy column: 1107/9180
dummy column: 11

dummy column: 1409/9180
dummy column: 1410/9180
dummy column: 1411/9180
dummy column: 1412/9180
dummy column: 1413/9180
dummy column: 1414/9180
dummy column: 1415/9180
dummy column: 1416/9180
dummy column: 1417/9180
dummy column: 1418/9180
dummy column: 1419/9180
dummy column: 1420/9180
dummy column: 1421/9180
dummy column: 1422/9180
dummy column: 1423/9180
dummy column: 1424/9180
dummy column: 1425/9180
dummy column: 1426/9180
dummy column: 1427/9180
dummy column: 1428/9180
dummy column: 1429/9180
dummy column: 1430/9180
dummy column: 1431/9180
dummy column: 1432/9180
dummy column: 1433/9180
dummy column: 1434/9180
dummy column: 1435/9180
dummy column: 1436/9180
dummy column: 1437/9180
dummy column: 1438/9180
dummy column: 1439/9180
dummy column: 1440/9180
dummy column: 1441/9180
dummy column: 1442/9180
dummy column: 1443/9180
dummy column: 1444/9180
dummy column: 1445/9180
dummy column: 1446/9180
dummy column: 1447/9180
dummy column: 1448/9180
dummy column: 1449/9180
dummy column: 14

dummy column: 1751/9180
dummy column: 1752/9180
dummy column: 1753/9180
dummy column: 1754/9180
dummy column: 1755/9180
dummy column: 1756/9180
dummy column: 1757/9180
dummy column: 1758/9180
dummy column: 1759/9180
dummy column: 1760/9180
dummy column: 1761/9180
dummy column: 1762/9180
dummy column: 1763/9180
dummy column: 1764/9180
dummy column: 1765/9180
dummy column: 1766/9180
dummy column: 1767/9180
dummy column: 1768/9180
dummy column: 1769/9180
dummy column: 1770/9180
dummy column: 1771/9180
dummy column: 1772/9180
dummy column: 1773/9180
dummy column: 1774/9180
dummy column: 1775/9180
dummy column: 1776/9180
dummy column: 1777/9180
dummy column: 1778/9180
dummy column: 1779/9180
dummy column: 1780/9180
dummy column: 1781/9180
dummy column: 1782/9180
dummy column: 1783/9180
dummy column: 1784/9180
dummy column: 1785/9180
dummy column: 1786/9180
dummy column: 1787/9180
dummy column: 1788/9180
dummy column: 1789/9180
dummy column: 1790/9180
dummy column: 1791/9180
dummy column: 17

dummy column: 2093/9180
dummy column: 2094/9180
dummy column: 2095/9180
dummy column: 2096/9180
dummy column: 2097/9180
dummy column: 2098/9180
dummy column: 2099/9180
dummy column: 2100/9180
dummy column: 2101/9180
dummy column: 2102/9180
dummy column: 2103/9180
dummy column: 2104/9180
dummy column: 2105/9180
dummy column: 2106/9180
dummy column: 2107/9180
dummy column: 2108/9180
dummy column: 2109/9180
dummy column: 2110/9180
dummy column: 2111/9180
dummy column: 2112/9180
dummy column: 2113/9180
dummy column: 2114/9180
dummy column: 2115/9180
dummy column: 2116/9180
dummy column: 2117/9180
dummy column: 2118/9180
dummy column: 2119/9180
dummy column: 2120/9180
dummy column: 2121/9180
dummy column: 2122/9180
dummy column: 2123/9180
dummy column: 2124/9180
dummy column: 2125/9180
dummy column: 2126/9180
dummy column: 2127/9180
dummy column: 2128/9180
dummy column: 2129/9180
dummy column: 2130/9180
dummy column: 2131/9180
dummy column: 2132/9180
dummy column: 2133/9180
dummy column: 21

dummy column: 2435/9180
dummy column: 2436/9180
dummy column: 2437/9180
dummy column: 2438/9180
dummy column: 2439/9180
dummy column: 2440/9180
dummy column: 2441/9180
dummy column: 2442/9180
dummy column: 2443/9180
dummy column: 2444/9180
dummy column: 2445/9180
dummy column: 2446/9180
dummy column: 2447/9180
dummy column: 2448/9180
dummy column: 2449/9180
dummy column: 2450/9180
dummy column: 2451/9180
dummy column: 2452/9180
dummy column: 2453/9180
dummy column: 2454/9180
dummy column: 2455/9180
dummy column: 2456/9180
dummy column: 2457/9180
dummy column: 2458/9180
dummy column: 2459/9180
dummy column: 2460/9180
dummy column: 2461/9180
dummy column: 2462/9180
dummy column: 2463/9180
dummy column: 2464/9180
dummy column: 2465/9180
dummy column: 2466/9180
dummy column: 2467/9180
dummy column: 2468/9180
dummy column: 2469/9180
dummy column: 2470/9180
dummy column: 2471/9180
dummy column: 2472/9180
dummy column: 2473/9180
dummy column: 2474/9180
dummy column: 2475/9180
dummy column: 24

KeyboardInterrupt: 

In [ ]:
 with open('./feats_dict_order4.pkl', 'rb') as f:
        z = pickle.load(f)

In [ ]:
z['train']['origin']

In [ ]:
class PipLine(PrePostProcess):
    def __init__(self):
        self.ALL_Features = dict()
        self.ALL_Features['train'] = dict()
        self.ALL_Features['valid'] = dict()
        
    def main(self, tra_x, tra_y, val_x, val_y, aff_cls, add_merge_box=False):
        '''
        cont_vars_list: a list of continues varible names;
        '''
        # 0. feature_dfs = [ori]
        self.ALL_Features['train']['origin']= tra_x
        self.ALL_Features['valid']['origin']= val_x
        
        for order in range(2,3):
            print('--------------------------order:{}--------------------'.format(order))
            cols = random.sample(tra_x.columns.tolist(), 8)
            tra_x, val_x = aff_cls.k_order_comb(tra_x, val_x, column_names=cols, order=order)
            
            if add_merge_box:
                tra_x, val_x =  aff_cls.merge_box(tra_x, val_x, columns=tra_x.columns, thresh=0.92)
                print('eval with only lgb_top_k feats after k_order_comb & dummy lgb & merge_box with order={}'.format(order))
                _ = aff_cls._auc_impo(tra_x.values, tra_y, val_x.values, val_y)

                            
            cols = tra_x.columns.tolist()
            tra_x_lgbk, val_x_lgbk = aff_cls.dummy_lgb_topk(tra_x, tra_y, val_x, val_y, 
                                          col_names_list=cols, max_unique=20000, top_k=100)
        
            print('| dummy_lgb_topk | k_order_comb | add_merge_box={}'.format(add_merge_box))
            _ = aff_cls._auc_impo(tra_x_lgbk.values, tra_y, val_x_lgbk.values, val_y)
            
            
            tra_x = PPP.add_null_rate(tra_x_lgbk, null_val=-999).join(tra_x, lsuffix ='order_{}_'.format(order))
            val_x = PPP.add_null_rate(val_x_lgbk, null_val=-999).join(val_x, lsuffix ='order_{}_'.format(order))
            tra_x = PPP.add_null_rate(tra_x_lgbk, null_val=-888).join(tra_x, lsuffix ='order_{}_'.format(order))
            val_x = PPP.add_null_rate(val_x_lgbk, null_val=-888).join(val_x, lsuffix ='order_{}_'.format(order))
            print('| dummy_lgb_topk | k_order_comb | null_val | add_merge_box={}'.format(add_merge_box))
            _ = aff_cls._auc_impo(tra_x.values, tra_y, val_x.values, val_y) 
            
           # 5. feature_dfs. append(new_cats)
            self.ALL_Features['train']['order_{}'.format(order)]= tra_x
            self.ALL_Features['valid']['order_{}'.format(order)]= val_x       
           
        # 5.根据分布一致性删除列；
        return self.ALL_Features

In [ ]:
# pipline = PipLine()
# aff = autofeat_draft(global_id='id')
# dic = pipline.main(aaa, tra_y, bbb, val_y, aff)